# GEMINI 2.5 PRO
## Refined mutation operator generated by Gemini 2.5 Pro during Stage 2 of the proposed refinement framework.

In [ ]:
# imports 
import os
import numpy as np
import ioh 
import pandas as pd
import inspect
import importlib
import re 

In [ ]:
# First, we define the DE class in order to use different mutation strategies
class DE:
    """
    Class implementing the Differential Evolution (DE) algorithm in a
    generic manner, allowing the mutation strategy to be easily exchanged.
    """
    def __init__(self,
                 mutation_function,  # Mutation as a function
                 pop_size: int = 50,
                 num_iterations: int = 2000,
                 F: float = 0.8,  # typical value: 0.8
                 CR: float = 0.2):  # typical value: 0.2
        """
        Constructor of the DE algorithm.

        Args:
            mutation_function: Function implementing the mutation strategy.
            pop_size (int): Population size.
            num_iterations (int): Number of generations/iterations.
            F (float): Scaling factor for mutation.
            CR (float): Crossover rate (binomial crossover probability).
        """
        self.mutation_function = mutation_function
        self.pop_size = pop_size
        self.num_iterations = num_iterations
        self.F = F
        self.CR = CR

    def __call__(self, problem: ioh.problem.RealSingleObjective) -> None:
        dim = problem.meta_data.n_variables
        low_b, high_b = problem.bounds.lb[0], problem.bounds.ub[0]

        population = np.random.uniform(low_b, high_b, (self.pop_size, dim))
        fitness = np.array([problem(ind) for ind in population])

        for _ in range(self.num_iterations):
            for i in range(self.pop_size):
                # a. Mutation (now calling the external mutation function)
                mutant = self.mutation_function(
                    i=i,
                    population=population,
                    fitness=fitness,
                    F=self.F,
                    low_b=low_b,
                    high_b=high_b
                )

                # b. Crossover (Binomial) - unchanged
                trial_vector = np.copy(population[i])
                j_rand = np.random.randint(0, dim)
                crossover_mask = np.random.rand(dim) < self.CR
                crossover_mask[j_rand] = True
                trial_vector[crossover_mask] = mutant[crossover_mask]
                
                # c. Selection - unchanged
                trial_fitness = problem(trial_vector)
                
                if trial_fitness <= fitness[i]:
                    population[i] = trial_vector
                    fitness[i] = trial_fitness

In [ ]:
# Mutation functions
# All comments and descriptions within this function were generated
# by the corresponding LLM model during the refinement stage.
def mutation_refined_gemini25pro(i: int, population: np.ndarray, fitness: np.ndarray, F: float,
    low_b: float, high_b: float) -> np.ndarray:
    """
    Strategy: "DE/current-to-mean-p-best/1 with Secondary Random Perturbation".

    This strategy is built upon the successful Cycle 9, which combined the
    'current-to-best' structure, adaptive damping, and an exploration guide
    defined by the centroid of the elite (x_mean_pbest). Although this
    combination proved to be highly effective on unimodal functions, the
    analysis of its regressions (e.g., Rastrigin, Gallagher) suggests that the
    search direction, being heavily averaged, may become overly conservative,
    thus losing the ability to effectively explore complex multimodal
    landscapes.

    To address this limitation, this new version introduces a second,
    smaller, and purely random exploration term. The main guided exploration
    component (x_mean_pbest - x_r1), which provides a robust search direction,
    is preserved. Additionally, a classical perturbation term (x_r2 - x_r3)
    is incorporated, scaled by a factor 'gamma' (e.g., 0.25) to ensure its
    influence remains limited.

    The objective is to reintroduce a controlled degree of diversity into the
    search process, allowing individuals to occasionally "jump" out of local
    optima basins of attraction without destabilizing the strong convergence
    provided by the main components of the strategy. This is expected to
    mitigate regressions on multimodal functions while preserving excellent
    performance on unimodal problems.

    The resulting mutation formula is:
    v = x_i + F_i*(x_best - x_i) + (d_i*F_i)*(x_mean_pbest - x_r1) + (gamma*F_i)*(x_r2 - x_r3).
    """
    pop_size = len(population)
    best_idx = np.argmin(fitness)

    # Adaptive F (successful component from previous cycles)
    F_i = np.random.normal(loc=F, scale=0.1)
    F_i = np.clip(F_i, 0.1, 1.0)

    # Rank-based adaptive damping (retained from Cycle 9)
    sorted_indices = np.argsort(fitness)
    ranks = np.empty_like(sorted_indices)
    ranks[sorted_indices] = np.arange(pop_size)
    rank_of_i = ranks[i]
    
    min_damping = 0.5
    max_damping = 1.0
    adaptive_damping_factor = (
        min_damping
        + (max_damping - min_damping) * (rank_of_i / (pop_size - 1))
    )

    # Mean-p-best guidance (retained from Cycle 9)
    p_value = 0.1
    num_p_best = max(2, int(pop_size * p_value))
    p_best_indices = sorted_indices[:num_p_best]
    x_mean_pbest = np.mean(population[p_best_indices], axis=0)

    # Selection of three random individuals for difference terms
    indices = list(range(pop_size))
    if i in indices:
        indices.remove(i)
    if best_idx in indices:
        indices.remove(best_idx)

    # Fallback if the population is too small
    if len(indices) < 3:
        # Revert to the Cycle 9 strategy if there are not enough individuals
        r1 = np.random.choice(indices)
        base_vector = population[i]
        exploitation_vector = population[best_idx] - population[i]
        guided_exploration_vector = x_mean_pbest - population[r1]

        mutant = (
            base_vector
            + F_i * exploitation_vector
            + (adaptive_damping_factor * F_i) * guided_exploration_vector
        )
    else:
        r1, r2, r3 = np.random.choice(indices, 3, replace=False)
    
        # --- Construction of the mutant vector ---
        base_vector = population[i]
        exploitation_vector = population[best_idx] - population[i]
        guided_exploration_vector = x_mean_pbest - population[r1]
        
        # New term: secondary random perturbation
        secondary_perturbation_vector = population[r2] - population[r3]
        gamma = 0.25  # Scaling factor for the secondary perturbation

        mutant = (
            base_vector
            + F_i * exploitation_vector
            + (adaptive_damping_factor * F_i) * guided_exploration_vector
            + (gamma * F_i) * secondary_perturbation_vector
        )

    return np.clip(mutant, low_b, high_b)


In [ ]:
def analyze_file_list(root_folder, file_list):
    all_results = []

    print(
        f"Analyzing {len(file_list)} files from folder: "
        f"'{os.path.abspath(root_folder)}'"
    )

    for file_name in file_list:
        # Build the full path to the file
        file_path = os.path.join(root_folder, file_name)
        
        try:
            # Extract information from the file name
            match_name = re.match(r"IOHprofiler_f(\d+)_(.*)\.info", file_name)
            if not match_name:
                continue
            
            func_id = int(match_name.group(1))
            func_name = match_name.group(2)

            # Read the file content
            with open(file_path, 'r') as f:
                content = f.read()
            
            # Extract results from repeated runs
            lines = content.splitlines()
            results_line = ""
            for i, current_line in enumerate(lines):
                if current_line.strip() == '%':
                    if i + 1 < len(lines):
                        results_line = lines[i + 1]
                        break
            
            if not results_line:
                raise ValueError(
                    "Results line not found after '%' delimiter"
                )

            matches_data = re.findall(
                r'(\d+)\s*:\s*(\d+)\s*\|\s*([\d.eE+-]+)',
                results_line
            )
            if not matches_data:
                raise ValueError("No result entries were found.")

            # Convert extracted data to numerical lists
            best_fitness_list = [float(m[2]) for m in matches_data]
            evaluations_list = [int(m[1]) for m in matches_data]
            
            # --- COMPUTATIONS ---
            # Compute mean, standard deviation, and median
            avg_fitness = np.mean(best_fitness_list)
            std_fitness = np.std(best_fitness_list)
            median_fitness = np.median(best_fitness_list)
            avg_evals = np.mean(evaluations_list)

            # --- STORE RESULTS ---
            all_results.append({
                "Function_ID": func_id,
                "Function_Name": func_name,
                "Avg_Fitness": avg_fitness,
                "Std_Fitness": std_fitness,
                "Median_Fitness": median_fitness,
                "Avg_Evals": avg_evals
            })
        except FileNotFoundError:
            print(f"- Warning: File '{file_path}' not found")
        except Exception as e:
            print(f"- Error processing file {file_name}: {e}")
    
    if not all_results:
        return None
        
    return pd.DataFrame(all_results)


In [ ]:

de_algorithm = DE(mutation_function=mutation_refined_gemini25pro, pop_size=50, F=0.8, CR=0.2)

experiment = ioh.Experiment(
    algorithm=de_algorithm,
    fids=range(1, 25), #Function range
    iids=[1], #Instances
    dims=[5], #Dimensions
    reps=30, #Number of Repetitions
    zip_output=False,
    folder_name="mutation_refined_gemini25pro",#Name of the output folder where results are stored
    old_logger=True
)

# To run the experiment, simply call it
experiment.run()

In [ ]:
# Analyze and display the results
if __name__ == "__main__":
    ROOT_FOLDER = "mutation_refined_gemini25pro" 

    files_to_analyze = [
        "IOHprofiler_f1_Sphere.info",
        "IOHprofiler_f2_Ellipsoid.info",
        "IOHprofiler_f3_Rastrigin.info",
        "IOHprofiler_f4_BuecheRastrigin.info",
        "IOHprofiler_f5_LinearSlope.info",
        "IOHprofiler_f6_AttractiveSector.info",
        "IOHprofiler_f7_StepEllipsoid.info",
        "IOHprofiler_f8_Rosenbrock.info",
        "IOHprofiler_f9_RosenbrockRotated.info",
        "IOHprofiler_f10_EllipsoidRotated.info",
        "IOHprofiler_f11_Discus.info",
        "IOHprofiler_f12_BentCigar.info",
        "IOHprofiler_f13_SharpRidge.info",
        "IOHprofiler_f14_DifferentPowers.info",
        "IOHprofiler_f15_RastriginRotated.info",
        "IOHprofiler_f16_Weierstrass.info",
        "IOHprofiler_f17_Schaffers10.info",
        "IOHprofiler_f18_Schaffers1000.info",
        "IOHprofiler_f19_GriewankRosenbrock.info",
        "IOHprofiler_f20_Schwefel.info",
        "IOHprofiler_f21_Gallagher101.info",
        "IOHprofiler_f22_Gallagher21.info",
        "IOHprofiler_f23_Katsuura.info",
        "IOHprofiler_f24_LunacekBiRastrigin.info"
    ]

    final_summary_df = analyze_file_list(root_folder=ROOT_FOLDER, file_list=files_to_analyze)

    print(final_summary_df)

In [ ]:
# Save the results to a .csv file
output_csv = "mutation_refined_gemini25pro.csv"
final_summary_df.to_csv(output_csv, index=False, encoding="utf-8-sig")
print(f"CSV file saved to: {output_csv}")